In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.cuda.is_available()

True

In [2]:
from src.vlstm_v4.interface import qkvkernel

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt21cu118/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt21cu118/lib


Using /home/max/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Creating extension directory /home/max/.cache/torch_extensions/py311_cu118/vlstm_v4...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu118/vlstm_v4/build.ninja...
Building extension module vlstm_v4...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] /home/max/miniconda3/envs/xlstmpt21cu118/bin/x86_64-conda-linux-gnu-c++ -MMD -MF interface.o.d -DTORCH_EXTENSION_NAME=vlstm_v4 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/TH -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/THC -isystem /home/max/miniconda3/envs/xlstmpt21cu118/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /home/max/myrepos/vlstm_cuda/src/vlstm_v4/interface.cc -o interface.o 
[2/3] /home/max/miniconda3/envs/xlstmpt21cu118/bin/nvcc  -ccbin /home/max/miniconda3

Loading extension module vlstm_v4...


### qkvtest

In [3]:
S = 16 # seq len
B = 1 # batch size
NH = 2 # num heads
DH = 4 # dim per head
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [9]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) #/ 100.
ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) #/ 10.
vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) #/ 10.
qs, qs.shape

(tensor([[[[  0.,   1.,   2.,   3.],
           [  4.,   5.,   6.,   7.],
           [  8.,   9.,  10.,  11.],
           [ 12.,  13.,  14.,  15.],
           [ 16.,  17.,  18.,  19.],
           [ 20.,  21.,  22.,  23.],
           [ 24.,  25.,  26.,  27.],
           [ 28.,  29.,  30.,  31.],
           [ 32.,  33.,  34.,  35.],
           [ 36.,  37.,  38.,  39.],
           [ 40.,  41.,  42.,  43.],
           [ 44.,  45.,  46.,  47.],
           [ 48.,  49.,  50.,  51.],
           [ 52.,  53.,  54.,  55.],
           [ 56.,  57.,  58.,  59.],
           [ 60.,  61.,  62.,  63.]],
 
          [[ 64.,  65.,  66.,  67.],
           [ 68.,  69.,  70.,  71.],
           [ 72.,  73.,  74.,  75.],
           [ 76.,  77.,  78.,  79.],
           [ 80.,  81.,  82.,  83.],
           [ 84.,  85.,  86.,  87.],
           [ 88.,  89.,  90.,  91.],
           [ 92.,  93.,  94.,  95.],
           [ 96.,  97.,  98.,  99.],
           [100., 101., 102., 103.],
           [104., 105., 106., 107.]

In [10]:
torch.set_printoptions(linewidth=200)
rs = qs @ ks.transpose(-1, -2)
rs, rs.shape

(tensor([[[[  6.,   6.,   6.,   6.,   6.,   6.,   6.,   6.,   6.,   6.,   6.,   6.,   6.,   6.,   6.,   6.],
           [ 22.,  22.,  22.,  22.,  22.,  22.,  22.,  22.,  22.,  22.,  22.,  22.,  22.,  22.,  22.,  22.],
           [ 38.,  38.,  38.,  38.,  38.,  38.,  38.,  38.,  38.,  38.,  38.,  38.,  38.,  38.,  38.,  38.],
           [ 54.,  54.,  54.,  54.,  54.,  54.,  54.,  54.,  54.,  54.,  54.,  54.,  54.,  54.,  54.,  54.],
           [ 70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.],
           [ 86.,  86.,  86.,  86.,  86.,  86.,  86.,  86.,  86.,  86.,  86.,  86.,  86.,  86.,  86.,  86.],
           [102., 102., 102., 102., 102., 102., 102., 102., 102., 102., 102., 102., 102., 102., 102., 102.],
           [118., 118., 118., 118., 118., 118., 118., 118., 118., 118., 118., 118., 118., 118., 118., 118.],
           [134., 134., 134., 134., 134., 134., 134., 134., 134., 134., 134., 134., 134., 134., 134., 134.],
           [150., 1

In [11]:
# pytorch version
rs = qs @ ks.transpose(-1, -2) @ vs
rs, rs.shape

(tensor([[[[  96.,   96.,   96.,   96.],
           [ 352.,  352.,  352.,  352.],
           [ 608.,  608.,  608.,  608.],
           [ 864.,  864.,  864.,  864.],
           [1120., 1120., 1120., 1120.],
           [1376., 1376., 1376., 1376.],
           [1632., 1632., 1632., 1632.],
           [1888., 1888., 1888., 1888.],
           [2144., 2144., 2144., 2144.],
           [2400., 2400., 2400., 2400.],
           [2656., 2656., 2656., 2656.],
           [2912., 2912., 2912., 2912.],
           [3168., 3168., 3168., 3168.],
           [3424., 3424., 3424., 3424.],
           [3680., 3680., 3680., 3680.],
           [3936., 3936., 3936., 3936.]],
 
          [[4192., 4192., 4192., 4192.],
           [4448., 4448., 4448., 4448.],
           [4704., 4704., 4704., 4704.],
           [4960., 4960., 4960., 4960.],
           [5216., 5216., 5216., 5216.],
           [5472., 5472., 5472., 5472.],
           [5728., 5728., 5728., 5728.],
           [5984., 5984., 5984., 5984.],
           [6

In [12]:
# cuda kernel
rs = qkvkernel(mat_Q=qs, mat_K=ks, mat_V=vs)
rs, rs.shape

before kernel dispatch - bfloat16!
B: 1, NH: 2, S: 16, DH: 4
blocksxy: 1-1, threads: 4-4
In Kernel: gdim.x: 1, gdim.y: 1, gdim.z: 1, bdim.x: 4, bdim.y: 4
In Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4
qTIdx=0|kvTIdx=0: qTile[3][3] = 15.000000
qTIdx=0|kvTIdx=0: kTile[0][3] = 1.000000
qTIdx=0|kvTIdx=0: sTile[3][0](3) = 54.000000
qTIdx=0|kvTIdx=0: qTile[7][3] = 31.000000
qTIdx=0|kvTIdx=0: kTile[0][3] = 1.000000
qTIdx=0|kvTIdx=0: sTile[7][0](3) = 118.000000
qTIdx=1|kvTIdx=0: qTile[3][3] = 47.000000
qTIdx=1|kvTIdx=0: kTile[0][3] = 1.000000
qTIdx=1|kvTIdx=0: sTile[3][0](3) = 182.000000
qTIdx=1|kvTIdx=0: qTile[7][3] = 63.000000
qTIdx=1|kvTIdx=0: kTile[0][3] = 1.000000
qTIdx=1|kvTIdx=0: sTile[7][0](3) = 246.000000
qTIdx=0|kvTIdx=0: qTile[3][3] = 79.000000
qTIdx=0|kvTIdx=0: kTile[0][3] = 1.000000
qTIdx=0|kvTIdx=0: sTile[3][0](3) = 310.000000
qTIdx=0|kvTIdx=0: qTile[7][3] = 95.000000
qTIdx=0|kvTIdx=0: kTile[0][3] = 1.000000
qTIdx=0|kvTIdx=0: sTile[7][0](3) = 374.000000
qTIdx=1|kvTIdx=0: qTile

(tensor([[[[   96.,    96.,    96.,    96.],
           [  352.,   352.,   352.,   352.],
           [  608.,   608.,   608.,   608.],
           [  864.,   864.,   864.,   864.],
           [ 1120.,  1120.,  1120.,  1120.],
           [ 1376.,  1376.,  1376.,  1376.],
           [ 1632.,  1632.,  1632.,  1632.],
           [ 1888.,  1888.,  1888.,  1888.],
           [ 2240.,  2240.,  2240.,  2240.],
           [ 2752.,  2752.,  2752.,  2752.],
           [ 3264.,  3264.,  3264.,  3264.],
           [ 3776.,  3776.,  3776.,  3776.],
           [ 4288.,  4288.,  4288.,  4288.],
           [ 4800.,  4800.,  4800.,  4800.],
           [ 5312.,  5312.,  5312.,  5312.],
           [ 5824.,  5824.,  5824.,  5824.]],
 
          [[ 6464.,  6464.,  6464.,  6464.],
           [ 7232.,  7232.,  7232.,  7232.],
           [ 8000.,  8000.,  8000.,  8000.],
           [ 8768.,  8768.,  8768.,  8768.],
           [ 9536.,  9536.,  9536.,  9536.],
           [10304., 10304., 10304., 10304.],
       

### Matmul kernel Test from vlstm_v2

In [10]:
# from src.vlstm_v3.interface import testkernel, copykernel, mmkernelv1

In [7]:
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")

In [12]:
S = 8 # sequence length
DH = 8 # hidden size

In [13]:
matA = torch.arange((2*S * DH), device=DEVICE, dtype=DTYPE).reshape((2*S, DH))
matB = torch.ones((DH, S), device=DEVICE, dtype=DTYPE)
matA.shape, matB.shape, matA, matA.sum(-1)

(torch.Size([16, 8]),
 torch.Size([8, 8]),
 tensor([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.],
         [  8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.],
         [ 16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.],
         [ 24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.],
         [ 32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.],
         [ 40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.],
         [ 48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.],
         [ 56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.],
         [ 64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.],
         [ 72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.],
         [ 80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.],
         [ 88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.],
         [ 96.,  97.,  98.,  99., 100., 101., 102., 103.],
         [104., 105., 106., 107., 108., 109., 110., 111.],
         [112., 113., 114., 115., 116., 117., 118., 119.],
         [120., 121., 122., 123., 124., 125., 126., 127.]], device='cuda

In [14]:
# pytorch
pt_out = matA @ matB
pt_out, pt_out.shape

(tensor([[ 28.,  28.,  28.,  28.,  28.,  28.,  28.,  28.],
         [ 92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.],
         [156., 156., 156., 156., 156., 156., 156., 156.],
         [220., 220., 220., 220., 220., 220., 220., 220.],
         [284., 284., 284., 284., 284., 284., 284., 284.],
         [348., 348., 348., 348., 348., 348., 348., 348.],
         [412., 412., 412., 412., 412., 412., 412., 412.],
         [476., 476., 476., 476., 476., 476., 476., 476.],
         [540., 540., 540., 540., 540., 540., 540., 540.],
         [604., 604., 604., 604., 604., 604., 604., 604.],
         [668., 668., 668., 668., 668., 668., 668., 668.],
         [732., 732., 732., 732., 732., 732., 732., 732.],
         [796., 796., 796., 796., 796., 796., 796., 796.],
         [860., 860., 860., 860., 860., 860., 860., 860.],
         [924., 924., 924., 924., 924., 924., 924., 924.],
         [988., 988., 988., 988., 988., 988., 988., 988.]], device='cuda:0', dtype=torch.bfloat16),
 torch.Size([16

In [15]:
matA.is_contiguous(), matB.is_contiguous()

(True, True)

In [16]:
cu_out = mmkernelv1(mat_A=matA, mat_B=matB)
cu_out, cu_out.shape

NameError: name 'mmkernelv1' is not defined

In [ ]:
matA[9]

In [ ]:
matA[9].cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.bfloat16, device=torch.device('cuda:0')).cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.bfloat16, device=torch.device('cpu')).cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.float32, device=torch.device('cpu')).cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.float16, device=torch.device('cuda:0')).cumsum(-1)

In [8]:
# cu_out = mmkernelv2(mat_A=matA, mat_B=matB)
# cu_out, cu_out.shape

In [ ]:
# mat @ mat.T @ mat

### pytorch version

In [ ]:
H = 6 # hidden size
S = 5 # seq len
B = 1 # batch size
NH = 2 # num heads
DH = H // NH # dim per head
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")
assert H % NH == 0

In [ ]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ds = torch.rand((B, NH, S, S), device=DEVICE, dtype=DTYPE)

max_log_D, _ = torch.max(ds.view(B, NH, -1), dim=-1, keepdim=True)  # (B, NH, 1)
log_D_matrix_stabilized = ds - max_log_D.unsqueeze(-1)  # (B, NH, S, S) = (B, NH, S, S) - (B, NH, 1, 1)
D_matrix = torch.exp(log_D_matrix_stabilized)  # (B, NH, S, S)
mval = torch.exp(-max_log_D.unsqueeze(-1))